In [1]:
import altair as alt
import pandas as pd
import pickle
import os
import numpy as np
import util
import gloop_net_analysis
from gloop_net_analysis import run_J, plot_kill_curves, plot_lag_distributions
from gloop_net_ltee import competition_assay
from igraph import Graph, EdgeSeq
import plotly.graph_objects as go

In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def gen_time_stack_df(all_N):
    total_genomes = list(set.union(*[set(N.keys()) for N in all_N]))
    times = [len(all_N[i][next(iter(all_N[i]))]) for i in range(len(all_N))]
    total_time = sum(times)
    # Dataset
    a = np.zeros(shape=(total_time, len(total_genomes)))
    s = np.cumsum([0] + times)
    s = zip(s[:-1], s[1:])
    for N in all_N:
        cur_s, cur_e = next(s)
        for g in total_genomes:
            if g in N:
                a[cur_s:cur_e, g] = N[g]
    df = pd.DataFrame.from_dict(
        {
            "ID": np.repeat(total_genomes, total_time),
            "Time": np.tile(np.arange(total_time), len(total_genomes)),
            "Count": a.T.flatten()
        })

    df.to_csv("tst", index=False)
    return df

In [62]:

data = gen_time_stack_df(all_N)
# data = pd.read_csv("tst")
url = 'data.json'
data.to_json(url, orient='records')

alt.Chart(url).mark_area().encode(
    alt.X('Time:Q'),
    alt.Y('Count:Q', stack='center'),
    alt.Color('ID:N', scale=alt.Scale(scheme='category20'))
).interactive(bind_y=False).properties(
    width=850,
    height=400)
# alt.Chart(url).mark_area().encode(
#     alt.X('Time:Q'),
#     alt.Y('Count:Q', stack='center'),
#     alt.Color('Avg_lag:N', scale=alt.Scale(scheme='inferno'))
# ).interactive(bind_y=False).properties(width=800,
#     height=400)


alt.Chart(...)

# Experiment (18/05/2022) - Fitness predictors 

The entire lag distribution ($L(t)$) should be sufficient to determine (average) fitness in the LTEE experiment, in whichever way "fitness" is defined. However, it may be that there are more compact predictors (say a few scalar functions of the lag distribution).

## Fitness
The fitness of a given genotype (A RCCN connection matrix $J$) can be defined either via copmetition assays (with ancestor. More complex competition assays will be considered later), or simply as the final fraction of said genotype in the end of the LTEE experiment.

## Predictors
Reasonable predictors are the mean and min of $L(t)$ (expected to be relevant for low/high values of the growth rate $\mu$, respectively), and the integralag 
$$\phi_{\mu}\left(L_{i},t\right)=\int_{\tau=0}^{t}\left(\exp\left[\mu\left(t-\tau\right)\right]+L_{i}\left(\tau\right)\right)d\tau$$

$$\phi_{\mu}\left(L_{i}\right)=\lim_{t\to\infty}\frac{\phi_{\mu}\left(L_{i},t\right)}{\sum_{j}\phi_{\mu}\left(L_{j},t\right)}$$

## Experiment
Numerous runs of an LTEE experiment starting with a single RCCN connection matrix and allowing for mutants to arise. 

The number of iterations $T$ will be fixed and so will the bottleneck *size* (it is also the bottleck in the sense of being the most expensive computationally, and thus rate limiting for the entire experiment). The mutation rate $\lambda$ will be fixed to allow for a managable number of mutants.

The bottleneck fraction $\rho$ and the growth rate $\mu$ will vary (TODO find appropriate ranges), for each combination of $\rho$ and $\mu$ the following will be recorded:

1. Fitness of all strains, as measured by:

    (a) Their fraction of the final population

    (b) Competition assays with the ancestor
    

2. The lag distribution (histogram, in practice) of each strain ($L\left(t\right)$), together with the following potential fitness predictors

    (a) The mean lag $\left\langle L\left(t\right)\right\rangle$

    (b) The “fast” fraction $L\left(0\right)$ (or maybe $L\left(0:t_{min}\right)$?)

    (c) The integralag function $\phi\left(L\right)$


The results of interest will be 

1. Predictability of fitness values from the predictors mentioned above as a function of $\mu,\rho$

2. The best solution found as a function of $\mu,\rho$





In [2]:
def retrieve_mutant(mutant_dict, mutant_idx, ancestor_J):
    cur_idx = mutant_idx
    J = np.copy(ancestor_J)
    trace = [cur_idx]
    while cur_idx != 0:
        cur_idx = mutant_dict[cur_idx]["parent_id"]
        trace = [cur_idx] + trace

    for m in trace[1:]:
        mutated_node = mutant_dict[m]["mutated_node"]
        mutated_connections = mutant_dict[m]["mutated_connections"]
        row_col_mutation = 0 if mutant_dict[m]["row_col_mutation"]=="row" else 1
        if row_col_mutation:
            J[:, mutated_node] = mutated_connections
        else:
            J[mutated_node, :] = mutated_connections
    return J, len(trace)

topology, ancestor_J = np.load("topologies_and_Js\\topology_1.npy"), np.load("topologies_and_Js\\topology_1_J_1.npy")


In [11]:
import matplotlib.pyplot as plt
def fitness_f(J, mu):
    Ns = competition_assay(Js=[ancestor_J, J], topology=topology, T_w=100, init_size=1000, final_size=500_000, growth_rate=mu)
    return Ns[1] / Ns[0]


folder = r"C:\Users\Owner\PycharmProjects\GloopNetEvolution\experiments\LTEE\2022_06_07_19_43_00_T_w=2000_diff_max_time"

load_from_exp = lambda exp_name, file: pickle.load(
    open(fr"{folder}\{exp_name}\{file}", 'rb'))
save_to_exp = lambda exp_name, file, to_save: pickle.dump(to_save,
    open(fr"{folder}\{exp_name}\{file}", 'wb'))
T_w = 2000


for i, exp_name in enumerate(os.listdir(fr"{folder}")):
    # mu = float(exp_name.split("_")[-2].split("=")[1])
    final_N = load_from_exp(exp_name, "final_N")
    mutant_dict = load_from_exp(exp_name, "mutant_dict")
    # plot_mutant_tree(mutant_dict, topology, final_N)
    mut_idx = max(final_N[-1], key=lambda x: [final_N[-1][x]])
    mut_J, gen = (ancestor_J, 1) if mut_idx==0 else retrieve_mutant(mutant_dict, mut_idx, ancestor_J)

    try:
        lag_times = load_from_exp(exp_name, f"lag_times_mut_{mut_idx}")
        mean_mag_records = load_from_exp(exp_name, f"mean_mag_records_mut_{mut_idx}")
    except FileNotFoundError:
        lag_times = run_J(topology=topology, J=mut_J, n_realizations_per_J=10000,
                          # T_ws=util.T_WS, record_mag=False)
                          T_ws=[100, 2000], record_mag=False)
                          # T_ws=[2000], record_mag=False)
        _, mag_records = run_J(topology=topology, J=mut_J, n_realizations_per_J=100,
                          # T_ws=util.T_WS, record_mag=False)
                          T_ws=[T_w], record_mag=True)
        mean_mag_records = np.mean(mag_records[T_w], axis=-1)
        save_to_exp(exp_name, f"lag_times_mut_{mut_idx}", lag_times)
        save_to_exp(exp_name, f"mean_mag_records_mut_{mut_idx}", mean_mag_records)  # Save only mean!


    plot_kill_curves(lag_times, title=f"Kill curve - {exp_name}\n Mutant {mut_idx} (gen {gen})", save_dir=fr"figure_dump/kill_curve_{exp_name}_mut_{mut_idx}")
    plot_lag_distributions({k: v for k, v in lag_times.items() if k == T_w}, x_max=150, y_max=300, title=f"Lag time distribution - {exp_name}\n Mutant {mut_idx} (gen({gen}))", save_dir=fr"figure_dump/lag_dist_{exp_name}_mut_{mut_idx}")
    plot_mag_trace(mean_mag_records, save_dir=fr"figure_dump/trace_{exp_name}_mut_{mut_idx}_",
                       lag_start_time =util.STABLE_T[1] + T_w)




fig saved to figure_dump/kill_curve_2022_06_07_20_24_41_1K_to_50K_T_w=2000_mu=0.100_max_time=50_i=0_mut_0.png
fig saved to figure_dump/lag_dist_2022_06_07_20_24_41_1K_to_50K_T_w=2000_mu=0.100_max_time=50_i=0_mut_0.png
fig saved to figure_dump/trace_2022_06_07_20_24_41_1K_to_50K_T_w=2000_mu=0.100_max_time=50_i=0_mut_0_.png
fig saved to figure_dump/kill_curve_2022_06_07_21_06_15_1K_to_50K_T_w=2000_mu=0.100_max_time=100_i=0_mut_70.png
fig saved to figure_dump/lag_dist_2022_06_07_21_06_15_1K_to_50K_T_w=2000_mu=0.100_max_time=100_i=0_mut_70.png
fig saved to figure_dump/trace_2022_06_07_21_06_15_1K_to_50K_T_w=2000_mu=0.100_max_time=100_i=0_mut_70_.png
fig saved to figure_dump/kill_curve_2022_06_07_21_46_18_1K_to_50K_T_w=2000_mu=0.100_max_time=None_i=0_mut_95.png
fig saved to figure_dump/lag_dist_2022_06_07_21_46_18_1K_to_50K_T_w=2000_mu=0.100_max_time=None_i=0_mut_95.png
fig saved to figure_dump/trace_2022_06_07_21_46_18_1K_to_50K_T_w=2000_mu=0.100_max_time=None_i=0_mut_95_.png
fig saved to 

In [3]:
def plot_mag_trace(mag_records, save_dir, lag_start_time):
    with plt.style.context("dark_background"):
        fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    ax.set(ylim=(-0.2, 0.6), xlim=(lag_start_time, lag_start_time+800))
    ax.axhline(y=0, lw=1)
    for realization_i in range(len(mag_records)):
        trace = mag_records[realization_i]
        crossing_idx = np.argmax(trace[lag_start_time:]<=0) + lag_start_time
        p = ax.plot(np.arange(len(trace))[lag_start_time:crossing_idx+5], trace[lag_start_time:crossing_idx+5], lw=0.2)
        ax.axvline(x=crossing_idx, c=p[0].get_c(), ymax=0.25, lw=1)
    # ax.plot(np.arange(len(mag_records)), mag_records, lw=0.2)
    util.save_fig(save_dir, figure_dump=False, facecolor="k")



In [4]:
def plot_mutant_tree(mutant_dict, topology, final_N, title="Mutant Tree"):
    nr_vertices = len(mutant_dict) + 1
    nodes = [0] + list(mutant_dict.keys())
    edges = []


    for i in nodes[1:]:
        edges += [(mutant_dict[i]["parent_id"], i)]
    edges = edges[::-1]

    G = Graph(edges) # 2 stands for children number

    lay = G.layout_reingold_tilford(mode="in", root=0)

    position = {k: lay[k] for k in range(nr_vertices)}
    Y = [lay[k][1] for k in range(nr_vertices)]
    M = max(Y)

    # es = EdgeSeq(G) # sequence of edges
    E = [e.tuple for e in G.es] # list of edges

    L = len(position)
    Xn = [position[k][0] for k in range(L)]
    Yn = [2*M-position[k][1] for k in range(L)]
    Xe = []
    Ye = []
    for edge in E:
        Xe+=[position[edge[0]][0],position[edge[1]][0], None]
        Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]


    # final_N_tot = sum(final_N[-1].values())
    get_final_conc = lambda i: int(final_N[-1][i]) if i in final_N[-1] else 0
    label_str = lambda i: f"{i}: (f_conc={get_final_conc(i):.0f}) Mutated {mutant_dict[i]['row_col_mutation']} #{mutant_dict[i]['mutated_node']} (size {topology[mutant_dict[i]['mutated_node']]})"
    descriptions = [f"Ancestor: (f_conc={get_final_conc(0):.0f})"] + [label_str(i) for i in nodes[1:]]

    mutated_cycle_size = [0] + [topology[mutant_dict[i]["mutated_node"]] for i in mutant_dict]
    final_conc = [get_final_conc(0)] + [get_final_conc(i) for i in mutant_dict]


    def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
        L=len(pos)
        if len(text)!=L:
            raise ValueError('The lists pos and text must have the same len')
        annotations = []
        for k in range(L):
            annotations.append(
                dict(
                    text=text[k], # or replace labels with a different list for the text within the circle
                    x=pos[k][0], y=2*M-position[k][1],
                    xref='x1', yref='y1',
                    font=dict(color=font_color, size=font_size),
                    showarrow=False)
            )
        return annotations


    fig = go.Figure()
    fig.add_trace(go.Scatter(x=Xe,
                       y=Ye,
                       mode='lines',
                       line=dict(color='rgb(210,210,210)', width=1),
                       hoverinfo='none'
                       ))
    fig.add_trace(go.Scatter(x=Xn,
                      y=Yn,
                      mode='markers',
                      name='bla',
                      marker=dict(symbol='circle-dot',
                                    # size=18,
                                    size=5*(np.log(np.array(mutated_cycle_size)+1)+1),
                                    # color=np.arange(nr_vertices),
                                    # color=mutated_cycle_size,
                                    color=final_conc,
                                    # color=[f"rgb({int(256*i/nr_vertices)},{int(256*i/nr_vertices)},{int(256*i/nr_vertices)})" for i in range(nr_vertices)],    #'#DB4551',
                                  # color_continuous_scale=["red", "green", "blue"],
                                    line=dict(color='rgb(50,50,50)', width=1)
                                    ),
                      text=descriptions,
                      hoverinfo='text',
                      opacity=0.8
                      ))

    axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                )

    fig.update_layout(title=title,
                  annotations=make_annotations(position, nodes),
                  font_size=12,
                  showlegend=False,
                  xaxis=axis,
                  yaxis=axis,
                  margin=dict(l=40, r=40, b=85, t=100),
                  hovermode='closest',
                  plot_bgcolor='rgb(248,248,248)'
                  )
    fig.show()